In [18]:
test_data = pd.read_excel('input_data.xlsx')
test_X = copy.deepcopy(test_data['TEXT'])

for i in range(len(test_X)):
    test_X[i] = test_X[i].replace(' ', '');    test_X[i] = test_X[i].replace(',', '');    test_X[i] = test_X[i].replace('.', '');    test_X[i] = test_X[i].replace(';', '');    test_X[i] = test_X[i].replace('~', '');    test_X[i] = test_X[i].replace('❤', '');    test_X[i] = test_X[i].replace('ㆍ', '');    test_X[i] = test_X[i].replace('_', '');    test_X[i] = test_X[i].replace('-', '');    test_X[i] = test_X[i].replace('(', '');    test_X[i] = test_X[i].replace(')', '');    test_X[i] = test_X[i].replace(':', '');    test_X[i] = test_X[i].replace('?', '');    test_X[i] = test_X[i].replace('!', ''); test_X[i] = test_X[i].replace('ᆢ', '')

test_X = typo_detection(test_X)

def test_feature_creation(sel_col):
    test_feature = pd.DataFrame(index = test_X, columns=sel_col)
    for i in range(len(test_feature.index)):
        for k in range(len(sel_col)):
            if (test_feature.index[i].find(sel_col[k]) != -1):
                test_feature.iloc[i][k] = 1
            else:
                test_feature.iloc[i][k] = 0
    return test_feature

test_feature_bin = test_feature_creation(sel_col_bin)
test_feature_cat_one = test_feature_creation(sel_col_cat_one)
test_feature_cat_two = test_feature_creation(sel_col_cat_two)

for i in test_feature_bin.columns:
    test_feature_bin[i] = pd.to_numeric(test_feature_bin[i])
for i in test_feature_cat_one.columns:
    test_feature_cat_one[i] = pd.to_numeric(test_feature_cat_one[i])
for i in test_feature_cat_two.columns:
    test_feature_cat_two[i] = pd.to_numeric(test_feature_cat_two[i])

In [19]:
# bin
svm_bin_pred_test = svm_bin.predict(test_feature_bin)
lgbc_bin_pred_test = lgbc_bin.predict(test_feature_bin)
xgbc_bin_pred_test = xgbc_bin.predict(test_feature_bin)

new_test_bin_feature = np.array([svm_bin_pred_test, lgbc_bin_pred_test, xgbc_bin_pred_test])
new_test_bin_feature = np.transpose(new_test_bin_feature)

test_final_bin = cbc_bin.predict(new_test_bin_feature)
    
#cat1
svm_cat_one_pred_test = svm_cat_one.predict(test_feature_cat_one)
lgbc_cat_one_pred_test = lgbc_cat_one.predict(test_feature_cat_one)
xgbc_cat_one_pred_test = xgbc_cat_one.predict(test_feature_cat_one)

new_test_cat_one_feature = np.array([svm_cat_one_pred_test, lgbc_cat_one_pred_test, xgbc_cat_one_pred_test])
new_test_cat_one_feature = np.transpose(new_test_cat_one_feature)

test_final_cat_one = cbc_cat_one.predict(new_test_cat_one_feature)

#cat2
svm_cat_two_pred_test = svm_cat_two.predict(test_feature_cat_two)
lgbc_cat_two_pred_test = lgbc_cat_two.predict(test_feature_cat_two)
xgbc_cat_two_pred_test = xgbc_cat_two.predict(test_feature_cat_two)

new_test_cat_two_feature = np.array([svm_cat_two_pred_test, lgbc_cat_two_pred_test, xgbc_cat_two_pred_test])
new_test_cat_two_feature = np.transpose(new_test_cat_two_feature)

test_final_cat_two = cbc_cat_two.predict(new_test_cat_two_feature)

In [20]:
test_result = []
for i in range(len(test_feature_bin.index)):
    if le_bin.inverse_transform(test_final_bin)[i] == '중립' or le_bin.inverse_transform(test_final_bin)[i] == '폐기':
        test_result.append(res_temp)
        continue
        
    res_temp = le_bin.inverse_transform(test_final_bin)[i] + '>' + le_cat_one.inverse_transform(test_final_cat_one)[i] + le_cat_two.inverse_transform(test_final_cat_two)[i]
    
    if res_temp.find('고객서비스') != -1:
        if res_temp.find('>상담시스템') == -1 and res_temp.find('>상담원') == -1:
            res_temp = res_temp[:res_temp.find('고객서비스')+5]
    if res_temp.find('삼성카드') != -1:
        if res_temp.find('>상담시스템') != -1 or res_temp.find('>상담원') != -1: 
            res_temp = res_temp[:res_temp.find('삼성카드')+4]
            
    if res_temp.find('>없음') != -1:
        res_temp = res_temp[:res_temp.find('>없음')]
    if res_temp.find('없음>') != -1:
        res_temp = res_temp[:res_temp.find('없음>')]
    if res_temp.find('없음') != -1:
        res_temp = res_temp[:res_temp.find('없음')]
    if res_temp.find('기타>') != -1:
        res_temp = res_temp[0:5]
        
    if res_temp == '칭찬' or res_temp =='불만':
        res_temp = res_temp + '>기타'
    
    test_result.append(res_temp)

for i in range(len(test_data['INT'])):
    test_data['INT'].iloc[i] = test_result[i]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# 결과출력

In [22]:
test_data.to_excel("result_data.xlsx", index=False, encoding='utf-8-sig')